# Gaussian Hill Diffusion


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import matplotlib.animation as animation
from IPython.display import HTML
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
rcParams['figure.figsize'] = (12, 6)

In [ ]:
from fenics import *
import numpy as np

### Formulation

The diffusion of a field $u$ is given by the unsteady heat equation

\begin{equation}
\frac{\partial u}{\partial t} = \nabla^2 u + f \qquad \text{in} \,\, \Omega\times (0, T],
\end{equation}

with boundary and inital conditions

\begin{equation}
\begin{split}
u = & u_D \qquad \text{on} \,\, \partial\Omega \times(0, T],\\
u = & u_0 \qquad \text{at} \,\, t=0.
\end{split}
\end{equation}

Here we will solve the diffusion of a Gaussian hill defined as

\begin{equation}
u_0(x, y) = e^{(-ax^2 - ay^2)},
\end{equation}

for $a=2$ on the domain $\Omega$: $[-2, 2]^2 \in \mathbb{R}^2$. We will use homogenous Dirichlet boundary conditions, implying that $u_D=0$.


### Variational Form and Time discretization

We will discretize the solution in time using a backward Euler method. This transforms your problem is a sequanece of stationary problems, where is stationary problem is a variational problem.

the discretized unsteday heat equations is (using Backward Euler)

\begin{equation}
u^{n+1} -\Delta t\nabla^2u^{n+1} = u^{n} + \Delta t f^{n+1}, \,\,\,\, n=1, 2,\cdots
\end{equation}

with

\begin{equation}
u^{0} =u_0.
\end{equation}

This can be expressed in its weak form as

\begin{equation}
\int_\Omega (u^{n+1}v + \Delta t\nabla u^{n+1} \cdot \nabla v)\,dx = \int_\Omega (u^n + \Delta t f^{n+1})v\,dx,
\end{equation}
or simply as
\begin{equation}
a(u^{n+1}, v) = L_{n+1}(v).
\end{equation}

This is the variational problem that must be solved at each time-step. In addition to this we also need to interpolate the initial condition on our function space (interpolation approach).

In [ ]:
T = 2.0                  # final time
num_steps = 50           # number of time steps
dt = T / num_steps       # time step

### Domain and Mesh

Here we will solve the problem on the domain $\Omega$ defined earlier. We will use 32 nodes in both $x$ and $y$ direction. 

(Hint: us the `RectangleMesh()` function with `Point(x, y)`.)

In [ ]:
nx = ny = 32       # 32 grid point in each direction
mesh = RectangleMesh(Point(-2, -2), Point(2, 2), nx, ny)
V = FunctionSpace(mesh, 'Lagrange', 1)
plot(mesh);

### Boundary Conditions

The homogenous Dirichlet boundary conditions are straight forward to apply to the function space $V$.

In [ ]:
bc = DirichletBC(V, Constant(0), "on_boundary")

The inital value $u_0$ must then be defined.

(Hint: recall that when defining an expression, any variable must be explicitly defined in the function `Expression()`.)

In [ ]:
u_0 = Expression('exp(-a*x[0]*x[0] - a*x[1]*x[1])',
                 degree=2, a=2)

Now we can interpolate the initial solution on the domain (via the function space $V$) to get our $u^n$

In [ ]:
u_n = interpolate(u_0, V)

Define the trial ($u$) and test ($v$) functions

In [ ]:
u = TrialFunction(V)
v = TestFunction(V)

### Source Term

We assumed no source term.

(Hint: can you really set `f = 0`?)

In [ ]:
f = Constant(0)

### Variational Form

Now we can write the problem into its variational form, as given above

In [ ]:
a = (u*v + dt*inner(grad(u), grad(v)))*dx
L = (u_n + dt*f)*v*dx

Before silving we just need to define a solution vector to hold the variable at each time-step, and initialise the solution time $t$.

In [ ]:
u_sol = Function(V)
t = 0
u_sols = []

Now we can loop over the number of steps, solving the variational problem at each time-step, as we did in the previous exercises. Finally, we must assign the value $u^{n+1}$ to $u^n$ before proceeding to the next time-step.

(Hint: use the function `.assign()` to assign the solution at time $n+1$ (`u_sol`) to the value at time $n$ (`u_n`).)

In [ ]:
for n in range(num_steps+1):
    
    # compute solution
    solve(a == L, u_sol, bc)
    
    # save the solution to an array
    u_sols.append(u_sol.compute_vertex_values(mesh))

    # update previous solution
    u_n.assign(u_sol)
# end

In [ ]:
xy = mesh.coordinates()
t = tri.Triangulation(xy[:, 0], xy[:, 1], mesh.cells())

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
tris = [ax.plot_trisurf(t, u_sols[0], cmap="viridis", vmax=0.8)]
ax.set_zlim(0, 0.8)

def animate(i):
    tris[0].remove()
    tris[0] = ax.plot_trisurf(t, u_sols[i+1], cmap="viridis", vmax=0.8)

anim = animation.FuncAnimation(fig, animate, frames=np.arange(0, 50, 1), interval=100)

HTML(anim.to_html5_video())